# Data here: 
## One and two day type 
## Three day type

# Day Type: One/two
## Create Date and Day of Week columns in the DF

In [1]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

load_dur = pd.read_csv('../outputs/load_long_format.csv')
solar_dur = pd.read_csv('../outputs/solar_long_format.csv')
wind_dur = pd.read_csv('../outputs/wind_long_format.csv')

## UNCOMMENT WHICH PROFILE TO BE USED
x = load_dur
x_name = 'load'
x_name2 = 'Load'
x_column = 'Load'

#x = solar_dur
#x_name = 'solar'
#x_name2 = 'Solar_Gen'
#choose TRG 
#x_column = 'TRG6'

#x = wind_dur
#x_name = 'wind'
#x_name2 = 'Wind_Gen'
#x_column = 'TRG4'

#this code creates an output directory in the parent director, if one does not exist yet
#Note: this is where all of the output files will be written, since outputs are large this saves space in git
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)

outputs_x = outputs_dir+'/'+x_name
if not os.path.exists(outputs_x):
    os.makedirs(outputs_x)
print('output files are written out in parent directory: '+outputs_x)

x = x[['Region','R_Group','R_Subgroup','Season','Month','Day','Hour',x_column]]
years = pd.read_csv('inputs/years.csv').dropna()

#print(x)

output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs
output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs/load


In [2]:
from itertools import cycle

#repeat the date for 24 times for each hour in the day
year_2016 = years['2016'].repeat(24).reset_index(drop=True)
year_2011 = years['2011'].repeat(24).reset_index(drop=True)
#print(year_2016)

#use itertools.cycle to make the date repeat until it reaches the end of the dataframe
year_2016 = cycle(year_2016)
year_2011 = cycle(year_2011)

#create temporary data frames where they will iterate through. Needed to achieve the correct length of the DF
temp2016 = x.loc[x['R_Group'] == 'ERC'].copy()
temp2011 = x.loc[x['R_Group'] != 'ERC'].copy()

#iterate through the date until the end of DF 
temp2016['Date'] = [next(year_2016) for year in range(len(temp2016))]
temp2011['Date'] = [next(year_2011) for year in range(len(temp2011))]
#print(temp2016)
#print(temp2011)

x2 = pd.concat([temp2016, temp2011], ignore_index=True)
#print(x2)

#convert date to a datetime type 
x2['Date'] = pd.to_datetime(x2['Date'])
x2['DOW'] = x2['Date'].dt.weekday

#check if it is a weekday or not 
weekday = pd.read_csv('inputs/weekday.csv')
x2 = pd.merge(x2,weekday,on='DOW',how='left')
#print(x2)

## Case 1: Monthly, single day type, 24 hours (288 segments)
#### Methodology: Using groupby function to group first by month, then by 24 hours

In [4]:
case1_x = x2.copy()

aggregations = {x_column:['count',sum,'mean']}
case1 = case1_x.groupby(['Region','Month','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Hour','Hour_Tot','Tot','Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
case1.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_1daytype_monthly_24hr.csv')
print()

case1_x2 = pd.merge(case1_x,case1,on=['Region','Month','Hour'],how='left')
case1_x2 = case1_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case1_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_1daytype_monthly_24hr.csv')

     Region  Month  Hour  Hour_Tot      Tot           Avg
0  ERC_REST      1     1        31  1042446  33627.290323
1  ERC_REST      1     2        31  1051661  33924.548387
2  ERC_REST      1     3        31  1079739  34830.290323
3  ERC_REST      1     4        31  1144492  36919.096774
4  ERC_REST      1     5        31  1239385  39980.161290
number of rows in dataset = 18144

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   

           Date  DOW  Weekday  Hour_Tot     Tot           Avg  
1545 2016-03-05    5    False        31  886708  28603.483871  
823  2016-02-04    3     True        30  870591  29019.700000  
1531 2016-03-04    4     True        31  886708  28603.483871  
number of rows in dataset = 551880


## Case 2: Season, single day-type, 24 hours (72 segments)
#### Methodology: Use groupby function to group by season and hour

In [5]:
case2_x = x2.copy()
case2_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case2_seasons = case2_seasons.drop(['bimonthly'], axis=1)
case2_seasons = case2_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case2_x = pd.merge(case2_x, case2_seasons, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case2 = case2_x.groupby(['Region','Season_Group','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season_Group','Hour','Hour_Tot','Tot','Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
case2.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_1daytype_season_24hr.csv')
print()

case2_x2 = pd.merge(case2_x,case2,on=['Region','Season_Group','Hour'],how='left')
case2_x2 = case2_x2.sort_values(['Region',x_column])
print(case2_x2.head(3))
print('number of rows in dataset =',case2_x2.shape[0])
case2_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_1daytype_season_24hr.csv')

     Region  Season_Group  Hour  Hour_Tot      Tot           Avg
0  ERC_REST             1     1        90  2928629  32540.322222
1  ERC_REST             1     2        90  2929331  32548.122222
2  ERC_REST             1     3        90  2978696  33096.622222
3  ERC_REST             1     4        90  3116896  34632.177778
4  ERC_REST             1     5        90  3346197  37179.966667
number of rows in dataset = 7560

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   

           Date  DOW  Weekday  Season_Group  Hour_Tot      Tot           Avg  
1545 2016-03-05    5    False             2       122  3603145  29533.975410  
823  2016-02-04    3     True             2       122  3587428  29405.147541  
1531 2016-03-04    4     True             2       

## Case 3: Monthly, weekend/weekday, 24 hours (576 segments)
#### Metholodogy: Use groupby function to group by month, then weekend/weekday, then by 24 hours

In [6]:
case3_x = x2.copy()

aggregations = {x_column:['count',sum,'mean']}
case3 = case3_x.groupby(['Region','Month','Weekday','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Month','Weekday','Hour','Hour_Tot','Tot','Avg']
print(case1.head())
print('number of rows in dataset =',case3.shape[0])
case3.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2daytype_monthly_24hr.csv')
print()

case3_x2 = pd.merge(case3_x,case3,on=['Region','Month','Weekday','Hour'],how='left')
case3_x2 = case3_x2.sort_values(['Region',x_column])
print(case3_x2.head())
print('number of rows in dataset =',case3_x2.shape[0])
case3_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2daytype_monthly_24hr.csv')

     Region  Month  Hour  Hour_Tot      Tot           Avg
0  ERC_REST      1     1        31  1042446  33627.290323
1  ERC_REST      1     2        31  1051661  33924.548387
2  ERC_REST      1     3        31  1079739  34830.290323
3  ERC_REST      1     4        31  1144492  36919.096774
4  ERC_REST      1     5        31  1239385  39980.161290
number of rows in dataset = 24507

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW  Weekday  Hour_Tot     Tot           Avg  
1545 2016-03-05    5    False        14  406333  29023.785714  
823  2016-02-04    3     True        30  870591  29019.7

## Case 4: Season, weekend/weekday, 24 hours (144 segments)
#### Methodology: groupby season, weekday, hour

In [7]:
case4_x = x2.copy()
case4_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case4_seasons = case4_seasons.drop(['bimonthly'], axis=1)
case4_seasons = case4_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case4_x = pd.merge(case4_x, case4_seasons, on='Month', how='left')
#print(case4_x)

aggregations = {x_column:['count',sum,'mean']}
case4 = case4_x.groupby(['Region','Season_Group','Weekday','Hour'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Season_Group','Weekday','Hour','Hour_Tot','Tot','Avg']
print(case4.head())
print('number of rows in dataset =',case4.shape[0])
case4.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2daytype_season_24hr.csv')
print()

case4_x2 = pd.merge(case4_x,case4,on=['Region','Season_Group','Weekday','Hour'],how='left')
case4_x2 = case4_x2.sort_values(['Region',x_column])
print(case4_x2.head())
print('number of rows in dataset =',case4_x2.shape[0])
case4_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2daytype_season_24hr.csv')

     Region  Season_Group  Weekday  Hour  Hour_Tot      Tot           Avg
0  ERC_REST             1    False     1        40  1226650  30666.250000
1  ERC_REST             1    False     2        30  1003087  33436.233333
2  ERC_REST             1    False     4        52  1741883  33497.750000
3  ERC_REST             1    False     5        18   669258  37181.000000
4  ERC_REST             1    False     7        64  2446079  38219.984375
number of rows in dataset = 12478

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW  Weekday  Season_Group  Hour_Tot      Tot           Avg  
1545 2016

## 4 hour interval day types

In [3]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/daytype_4hr.csv')
#print(interval_4hr)

x3 = pd.merge(x2,interval_4hr,on='Hour',how='left').rename(columns={'4-hr_Day':'4-hr'})
print(x3)

          Region R_Group R_Subgroup  Season  Month  Day  Hour   Load  \
0       ERC_REST     ERC       REST  winter      1    1     1  34807   
1       ERC_REST     ERC       REST  winter      1    2     1  34716   
2       ERC_REST     ERC       REST  winter      1    3     1  34736   
3       ERC_REST     ERC       REST  winter      1    4     1  35914   
4       ERC_REST     ERC       REST  winter      1    5     1  33845   
...          ...     ...        ...     ...    ...  ...   ...    ...   
551875   WECC_WY     WEC         WY  winter     12  361    24   1767   
551876   WECC_WY     WEC         WY  winter     12  362    24   1744   
551877   WECC_WY     WEC         WY  winter     12  363    24   1805   
551878   WECC_WY     WEC         WY  winter     12  364    24   1791   
551879   WECC_WY     WEC         WY  winter     12  365    24   1834   

             Date  DOW  Weekday  4-hr  
0      2016-01-01    4     True     1  
1      2016-01-01    4     True     1  
2      2016-01-

## Case 5: Monthly, single day type, 4 hour intervals (72 segments)
#### Methodology: use groupby by month, 4 hour intervals

In [5]:
case5_x = x3.copy()

aggregations = {x_column:['count',sum,'mean']}
case5 = case5_x.groupby(['Region','Month','Day','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Month','Day','4-hr','Hour_Tot','Tot','Avg']
print(case5.head())
print('number of rows in dataset =',case5.shape[0])
case5.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2daytype_month_4hr.csv')
print()

case5_x2 = pd.merge(case5_x,case5,on=['Region','Month','Day','4-hr'],how='left')
case5_x2 = case5_x2.sort_values(['Region',x_column])
print(case5_x2.head())
print('number of rows in dataset =',case5_x2.shape[0])
case5_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2daytype_month_4hr.csv')

     Region  Month  Day  4-hr  Hour_Tot     Tot       Avg
0  ERC_REST      1    1     1         4  139677  34919.25
1  ERC_REST      1    1     2         4  153285  38321.25
2  ERC_REST      1    1     3         4  160446  40111.50
3  ERC_REST      1    1     4         4  155314  38828.50
4  ERC_REST      1    1     5         4  162472  40618.00
number of rows in dataset = 137970

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW  Weekday  4-hr  Hour_Tot     Tot       Avg  
1545 2016-03-05    5    False     1         4  108862  27215.50  
823  2016-02-04    3     True     1         4  1110

## Case 6: Bi-monthly weekend/weekday day-types, 4-hour intervals
#### Methodology: use groupby function and bimonthly groups

In [6]:
case6_x = x3.copy()
case6_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case6_bimonth = case6_bimonth.drop(['seasonal'], axis=1)
case6_bimonth = case6_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case6_x = pd.merge(case6_x, case6_bimonth, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case6 = case6_x.groupby(['Region','Bimonth','Day','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Bimonth','Day','4-hr','Hour_Tot','Tot','Avg']
print(case6.head())
print('number of rows in dataset =',case6.shape[0])
case6.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2daytype_bimonth_4hr.csv')
print()

case6_x2 = pd.merge(case6_x,case6,on=['Region','Bimonth','Day','4-hr'],how='left')
case6_x2 = case6_x2.sort_values(['Region',x_column])
print(case6_x2.head())
print('number of rows in dataset =',case6_x2.shape[0])
case6_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2daytype_bimonth_4hr.csv')

     Region  Bimonth  Day  4-hr  Hour_Tot     Tot       Avg
0  ERC_REST        1    1     1         4  139677  34919.25
1  ERC_REST        1    1     2         4  153285  38321.25
2  ERC_REST        1    1     3         4  160446  40111.50
3  ERC_REST        1    1     4         4  155314  38828.50
4  ERC_REST        1    1     5         4  162472  40618.00
number of rows in dataset = 137970

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW  Weekday  4-hr  Bimonth  Hour_Tot     Tot       Avg  
1545 2016-03-05    5    False     1        2         4  108862  27215.50  
823  2016-02-04    3 

## Case 7: Season-based months, weekend/weekday day-types, 4-hour intervals
#### Methodology: groupby function and applied season groups

In [7]:
case7_x = x3.copy()
case7_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case7_seasons = case7_seasons.drop(['bimonthly'], axis=1)
case7_seasons = case7_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case7_x = pd.merge(case7_x, case7_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {x_column:['count',sum,'mean']}
case7 = case7_x.groupby(['Region','Season','Weekday','4-hr'],as_index=False).agg(aggregations)
case7.columns = case7.columns.droplevel(0)
case7.columns = ['Region','Season','Weekday','4-hr','Hour_Tot','Tot','Avg']
print(case7.head())
print('number of rows in dataset =',case7.shape[0])
case7.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_2daytype_season_4hr.csv')
print()

case7_x2 = pd.merge(case7_x,case7,on=['Region','Season','Weekday','4-hr'],how='left')
case7_x2 = case7_x2.sort_values(['Region',x_column])
print(case7_x2.head())
print('number of rows in dataset =',case7_x2.shape[0])
case7_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_2daytype_season_4hr.csv')

     Region    Season  Weekday  4-hr  Hour_Tot      Tot           Avg
0  ERC_REST  shoulder    False     1       115  3399710  29562.695652
1  ERC_REST  shoulder    False     2       144  4872764  33838.638889
2  ERC_REST  shoulder    False     3       155  5809365  37479.774194
3  ERC_REST  shoulder    False     4       115  4632532  40282.886957
4  ERC_REST  shoulder    False     5       144  5663089  39327.006944
number of rows in dataset = 2268

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW  Weekday  4-hr  Season_Group  Hour_Tot       Tot  \
1545 2016-03-05    5    False     1     

# Day type: Three (Weekday, Weekend, Peak Load)
## Find Peak Load Days in Each Month

In [3]:
#create temporary DF to find peak
test = x.copy()

#groupby region, month, and day to sum the total day
aggregations1 = {x_column:sum}
test_sum = test.groupby(['Region','Month','Day'],as_index=False).agg(aggregations1)
#test1.columns = test1.columns.droplevel(0)
test_sum.columns = ['Region','Month','Day','Tot']
#print(test_sum.head())
#print('number of rows in dataset =',test_sum.shape[0])

test3 = pd.merge(test,test_sum,on=['Region','Month','Day'],how='left')
#print(test3)

#groupby region and month to find maximum 
aggregations2 = {'Tot':max}
test_max = test_sum.groupby(['Region','Month'],as_index=False).agg(aggregations2)
test_max.columns = ['Region','Month','Max']
#print(test_max.head())

test4 = pd.merge(test3,test_max,on=['Region','Month'],how='left')
print(test4)

          Region R_Group R_Subgroup  Season  Month  Day  Hour   Load     Tot  \
0       ERC_REST     ERC       REST  winter      1    1     1  34807  917588   
1       ERC_REST     ERC       REST  winter      1    2     1  34716  863132   
2       ERC_REST     ERC       REST  winter      1    3     1  34736  934513   
3       ERC_REST     ERC       REST  winter      1    4     1  35914  960576   
4       ERC_REST     ERC       REST  winter      1    5     1  33845  916216   
...          ...     ...        ...     ...    ...  ...   ...    ...     ...   
551875   WECC_WY     WEC         WY  winter     12  361    24   1767   51381   
551876   WECC_WY     WEC         WY  winter     12  362    24   1744   49456   
551877   WECC_WY     WEC         WY  winter     12  363    24   1805   48194   
551878   WECC_WY     WEC         WY  winter     12  364    24   1791   47653   
551879   WECC_WY     WEC         WY  winter     12  365    24   1834   47420   

            Max  
0       1000605  
1  

In [4]:
x_peak = x2.copy()

x_peak = pd.merge(x_peak,test4,on=['Region','R_Group','R_Subgroup','Season','Month','Day','Hour',x_column],how='left')
x_peak = x_peak.rename(columns={'Weekday':'Day_Type'})

#Return True if the load total equals the day identified as the max
x_peak.loc[x_peak['Day_Type'] == True, 'Day_Type'] = 'Weekday'
x_peak.loc[x_peak['Day_Type'] == False, 'Day_Type'] = 'Weekend'
x_peak.loc[x_peak['Tot'] == x_peak['Max'], 'Day_Type'] = 'Peak'
x_peak = x_peak.drop(['Tot','Max'], axis=1)
print(x_peak)

          Region R_Group R_Subgroup  Season  Month  Day  Hour   Load  \
0       ERC_REST     ERC       REST  winter      1    1     1  34807   
1       ERC_REST     ERC       REST  winter      1    2     1  34716   
2       ERC_REST     ERC       REST  winter      1    3     1  34736   
3       ERC_REST     ERC       REST  winter      1    4     1  35914   
4       ERC_REST     ERC       REST  winter      1    5     1  33845   
...          ...     ...        ...     ...    ...  ...   ...    ...   
551875   WECC_WY     WEC         WY  winter     12  361    24   1767   
551876   WECC_WY     WEC         WY  winter     12  362    24   1744   
551877   WECC_WY     WEC         WY  winter     12  363    24   1805   
551878   WECC_WY     WEC         WY  winter     12  364    24   1791   
551879   WECC_WY     WEC         WY  winter     12  365    24   1834   

             Date  DOW Day_Type  
0      2016-01-01    4  Weekday  
1      2016-01-01    4  Weekday  
2      2016-01-01    4  Weekday  

## Case 1: Monthly, Weekend/weekday/peak day-types, 24 hours (864 segments)
#### Methodology: similar to two day type, just adding in peak day types to sort by

In [14]:
case1_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case1 = case1_x.groupby(['Region','Month','Day_Type','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Day_Type','Hour','Hour_Tot','Tot','Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
case1.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3daytype_monthly_24hr.csv')
print()

case1_x2 = pd.merge(case1_x,case1,on=['Region','Month','Day_Type','Hour'],how='left')
case1_x2 = case1_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case1_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3daytype_monthly_24hr.csv')

     Region  Month Day_Type  Hour  Hour_Tot    Tot      Avg
0  ERC_REST      1     Peak     1         1  38567  38567.0
1  ERC_REST      1     Peak     2         1  39107  39107.0
2  ERC_REST      1     Peak     3         1  40311  40311.0
3  ERC_REST      1     Peak     4         1  43115  43115.0
4  ERC_REST      1     Peak     5         1  47186  47186.0
number of rows in dataset = 42612

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
2233  ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1706  ERC_REST     ERC       REST  shoulder      3   72     3  27025   

           Date  DOW Day_Type  Hour_Counter  Hour_Tot     Tot           Avg  
2042 2016-03-05    5  Weekend          2043        13  376380  28952.307692  
2233 2016-02-04    3  Weekday          2234        29  837721  28886.931034  
1706 2016-03-04    4  Weekday          1707        17  480375  28257.352941  
numb

## Case 2: Season, weekend/weekday/peak day-types, 24-hours (216 segments)

In [15]:
case2_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case2 = case2_x.groupby(['Region','Season','Day_Type','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season','Day_Type','Hour','Hour_Tot','Tot','Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
case2.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3daytype_season_24hr.csv')
print()

case2_x2 = pd.merge(case2_x,case2,on=['Region','Season','Day_Type','Hour'],how='left')
case2_x2 = case2_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case2_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3daytype_season_24hr.csv')

     Region    Season Day_Type  Hour  Hour_Tot     Tot       Avg
0  ERC_REST  shoulder     Peak     1         4  132690  33172.50
1  ERC_REST  shoulder     Peak     2         4  130802  32700.50
2  ERC_REST  shoulder     Peak     3         4  131751  32937.75
3  ERC_REST  shoulder     Peak     4         4  137910  34477.50
4  ERC_REST  shoulder     Peak     5         4  150015  37503.75
number of rows in dataset = 12919

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
2233  ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1706  ERC_REST     ERC       REST  shoulder      3   72     3  27025   

           Date  DOW Day_Type  Hour_Counter  Hour_Tot     Tot           Avg  
2042 2016-03-05    5  Weekend          2043        13  376380  28952.307692  
2233 2016-02-04    3  Weekday          2234        29  837721  28886.931034  
1706 2016-03-04    4  Weekday          1707        1

## Case 3: Annual, weekend/weekday/peak day-types, 24-hours (72 segments)

In [16]:
case3_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case3 = case3_x.groupby(['Region','Day_Type','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Day_Type','Hour','Hour_Tot','Tot','Avg']
print(case3.head())
print('number of rows in dataset =',case3.shape[0])
case3.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3daytype_annual_24hr.csv')
print()

case3_x2 = pd.merge(case3_x,case3,on=['Region','Day_Type','Hour'],how='left')
case3_x2 = case3_x2.sort_values(['Region',x_column])
print(case3_x2.head(3))
print('number of rows in dataset =',case3_x2.shape[0])
case3_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3daytype_annual_24hr.csv')

     Region Day_Type  Hour  Hour_Tot     Tot           Avg
0  ERC_REST     Peak     1        12  455893  37991.083333
1  ERC_REST     Peak     2        12  450804  37567.000000
2  ERC_REST     Peak     3        12  454384  37865.333333
3  ERC_REST     Peak     4        12  473371  39447.583333
4  ERC_REST     Peak     5        12  505999  42166.583333
number of rows in dataset = 4536

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
2042  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
2233  ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1706  ERC_REST     ERC       REST  shoulder      3   72     3  27025   

           Date  DOW Day_Type  Hour_Counter  Hour_Tot      Tot           Avg  
2042 2016-03-05    5  Weekend          2043        92  2899625  31517.663043  
2233 2016-02-04    3  Weekday          2234       247  8133375  32928.643725  
1706 2016-03-04    4  Weekday          1707       261  8694501  33312.264368  
number 

## 4 hour interval

In [5]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/daytype_4hr.csv')

x_peak2 = pd.merge(x_peak,interval_4hr,on='Hour',how='left').rename(columns={'4-hr_Day':'4-hr'})
print(x_peak2)

          Region R_Group R_Subgroup  Season  Month  Day  Hour   Load  \
0       ERC_REST     ERC       REST  winter      1    1     1  34807   
1       ERC_REST     ERC       REST  winter      1    2     1  34716   
2       ERC_REST     ERC       REST  winter      1    3     1  34736   
3       ERC_REST     ERC       REST  winter      1    4     1  35914   
4       ERC_REST     ERC       REST  winter      1    5     1  33845   
...          ...     ...        ...     ...    ...  ...   ...    ...   
551875   WECC_WY     WEC         WY  winter     12  361    24   1767   
551876   WECC_WY     WEC         WY  winter     12  362    24   1744   
551877   WECC_WY     WEC         WY  winter     12  363    24   1805   
551878   WECC_WY     WEC         WY  winter     12  364    24   1791   
551879   WECC_WY     WEC         WY  winter     12  365    24   1834   

             Date  DOW Day_Type  4-hr  
0      2016-01-01    4  Weekday     1  
1      2016-01-01    4  Weekday     1  
2      2016-01-

## Case 4: Bi-monthly, weekend/weekday/peak day-types, 4-hour intervals

In [6]:
case4_x = x_peak2.copy()
case4_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case4_bimonth = case4_bimonth.drop(['seasonal'], axis=1)
case4_bimonth = case4_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case4_x = pd.merge(case4_x, case4_bimonth, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case4 = case4_x.groupby(['Region','Bimonth','Day_Type','4-hr'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Bimonth','Day_Type','4-hr','Hour_Tot','Tot','Avg']
print(case4.head())
print('number of rows in dataset =',case4.shape[0])
case4.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3daytype_bimonth_4hr.csv')
print()

case4_x2 = pd.merge(case4_x,case4,on=['Region','Bimonth','Day_Type','4-hr'],how='left')
case4_x2 = case4_x2.sort_values(['Region',x_column])
print(case4_x2.head())
print('number of rows in dataset =',case4_x2.shape[0])
case4_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3daytype_bimonth_4hr.csv')

     Region  Bimonth Day_Type  4-hr  Hour_Tot     Tot        Avg
0  ERC_REST        1     Peak     1         8  312443  39055.375
1  ERC_REST        1     Peak     2         8  366535  45816.875
2  ERC_REST        1     Peak     3         8  322104  40263.000
3  ERC_REST        1     Peak     4         8  297348  37168.500
4  ERC_REST        1     Peak     5         8  332533  41566.625
number of rows in dataset = 6804

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW Day_Type  4-hr  Bimonth  Hour_Tot      Tot           Avg  
1545 2016-03-05    5  Weekend     1        2        62  1790938

## Case 5: Season-based months, weekend/weekday/peak day-types, 4-hour intervals

In [7]:
case5_x = x_peak2.copy()
case5_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case5_seasons = case5_seasons.drop(['bimonthly'], axis=1)
case5_seasons = case5_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case5_x = pd.merge(case5_x, case5_seasons, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case5 = case5_x.groupby(['Region','Season_Group','Day_Type','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Season_Group','Day_Type','4-hr','Hour_Tot','Tot','Avg']
print(case5.head())
print('number of rows in dataset =',case5.shape[0])
case5.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3daytype_seasonbased_4hr.csv')
print()

case5_x2 = pd.merge(case5_x,case5,on=['Region','Season_Group','Day_Type','4-hr'],how='left')
case5_x2 = case5_x2.sort_values(['Region',x_column])
print(case5_x2.head())
print('number of rows in dataset =',case5_x2.shape[0])
case5_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3daytype_seasonbased_4hr.csv')

     Region  Season_Group Day_Type  4-hr  Hour_Tot     Tot           Avg
0  ERC_REST             1     Peak     1        12  504276  42023.000000
1  ERC_REST             1     Peak     2        12  583360  48613.333333
2  ERC_REST             1     Peak     3        12  527061  43921.750000
3  ERC_REST             1     Peak     4        12  478413  39867.750000
4  ERC_REST             1     Peak     5        12  530620  44218.333333
number of rows in dataset = 5639

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW Day_Type  4-hr  Season_Group  Hour_Tot       Tot  \
1545 2016-03-05    5  

## Case 6: Season, weekend/weekday/peak day-types, 4-hour intervals

In [9]:
case6_x = x_peak2.copy()

aggregations = {x_column:['count',sum,'mean']}
case6 = case6_x.groupby(['Region','Season','Day_Type','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Season','Day_Type','4-hr','Hour_Tot','Tot','Avg']
print(case6.head())
print('number of rows in dataset =',case6.shape[0])
case6.to_csv('../outputs/'+x_name+'/'+x_name+'_segments_3daytype_season_4hr.csv')
print()

case6_x2 = pd.merge(case6_x,case6,on=['Region','Season','Day_Type','4-hr'],how='left')
case6_x2 = case6_x2.sort_values(['Region',x_column])
print(case6_x2.head())
print('number of rows in dataset =',case6_x2.shape[0])
case6_x2.to_csv('../outputs/'+x_name+'/'+x_name+'_8760_3daytype_season_4hr.csv')

     Region    Season Day_Type  4-hr  Hour_Tot     Tot         Avg
0  ERC_REST  shoulder     Peak     1        16  533153  33322.0625
1  ERC_REST  shoulder     Peak     2        16  619476  38717.2500
2  ERC_REST  shoulder     Peak     3        16  707544  44221.5000
3  ERC_REST  shoulder     Peak     4        16  785332  49083.2500
4  ERC_REST  shoulder     Peak     5        16  743101  46443.8125
number of rows in dataset = 3402

        Region R_Group R_Subgroup    Season  Month  Day  Hour   Load  \
1545  ERC_REST     ERC       REST  shoulder      3   86     3  26989   
823   ERC_REST     ERC       REST  shoulder      4   94     2  27006   
1531  ERC_REST     ERC       REST  shoulder      3   72     3  27025   
1524  ERC_REST     ERC       REST  shoulder      3   65     3  27059   
794   ERC_REST     ERC       REST  shoulder      3   65     2  27069   

           Date  DOW Day_Type  4-hr  Hour_Tot       Tot           Avg  
1545 2016-03-05    5  Weekend     1       111   3268235  29